In [1]:
from pathlib import Path

import tqdm
from PIL import Image

In [2]:
TRAIN_IMAGE_DIR = "../data/images/train"
TRAIN_ANNOTATION_DIR = "../data/labels/yolo/train_labels"
VAL_IMAGE_DIR = "../data/images/validation"
VAL_ANNOTATION_DIR = "../data/labels/yolo/val_labels"

EXPORT_TRAIN_DIR = "../data/market1501/bounding_box_train"
EXPORT_TEST_DIR  = "../data/market1501/bounding_box_test"   # With label
EXPORT_QUERY_DIR = "../data/market1501/gt_query"  # With label

In [3]:
train_image_dir = Path(TRAIN_IMAGE_DIR)
train_annotation_dir = Path(TRAIN_ANNOTATION_DIR)
val_image_dir = Path(VAL_IMAGE_DIR)
val_annotation_dir = Path(VAL_ANNOTATION_DIR)

export_train_dir = Path(EXPORT_TRAIN_DIR)
export_test_dir = Path(EXPORT_TEST_DIR)
export_query_dir = Path(EXPORT_QUERY_DIR)

export_train_dir.mkdir(parents=True, exist_ok=True)
export_test_dir.mkdir(parents=True, exist_ok=True)
export_query_dir.mkdir(parents=True, exist_ok=True)

In [4]:
# Export cropped train data
for label_path in tqdm.tqdm(train_annotation_dir.glob("*.txt")):
    with open(label_path) as f:
        for idx, label_infos in enumerate(f.readlines()):
            [c, n_x, n_y, n_w, n_h] = map(float, label_infos.split(" "))

            # Load image
            img = Image.open(
                train_image_dir / label_path.name.replace(".txt", ".png")
            ).convert("RGB")

            # Convert yolo bbox to PIL bbox
            left = (n_x - n_w / 2) * img.width
            top = (n_y - n_h / 2) * img.height
            right = (n_x + n_w / 2) * img.width
            bottom = (n_y + n_h / 2) * img.height

            # Crop plushie
            cropped_img = img.crop((left, top, right, bottom))

            # Save cropped plushie
            cropped_img.save(
                export_train_dir / f"{int(c)}_c1s1_{label_path.stem}_{idx}.jpg"
            )

5664it [03:41, 25.57it/s]


In [5]:
suspect_file_ids = [607, 623, 131, 24, 47, 282, 129, 55, 125, 569, 58, 259, 290, 23, 302, 103, 264, 249, 308, 590, 44, 272, 621, 102, 615, 150, 291, 561, 262, 284, 582, 586, 589, 604, 113, 576, 572, 119, 260, 144, 38, 265, 612, 41, 271, 602, 27, 153, 112, 636, 157, 118, 571, 266, 578, 17, 251, 279, 18, 159, 301, 306, 609, 285, 567, 6, 281, 152, 634, 49, 87, 599, 632, 84, 73, 134, 14, 34, 562, 155, 48, 581, 151, 316, 105, 287, 250, 294, 631, 313, 3, 610, 588, 78, 598, 85, 277, 62, 603, 109]

# Export cropped test data
for label_path in tqdm.tqdm(val_annotation_dir.glob("*.txt")):
    with open(label_path) as f:
        for idx, label_infos in enumerate(f.readlines()):
            [c, n_x, n_y, n_w, n_h] = map(float, label_infos.split(" "))

            # Load image
            img = Image.open(
                val_image_dir / label_path.name.replace(".txt", ".png")
            ).convert("RGB")

            # Convert yolo bbox to PIL bbox
            left = (n_x - n_w / 2) * img.width
            top = (n_y - n_h / 2) * img.height
            right = (n_x + n_w / 2) * img.width
            bottom = (n_y + n_h / 2) * img.height

            # Crop plushie
            cropped_img = img.crop((left, top, right, bottom))

            # Save cropped plushie
            if int(label_path.stem.replace("image_", "")) in suspect_file_ids:
                # Save to suspect dir
                cropped_img.save(
                    export_query_dir / f"{int(c)}_c1s1_{label_path.stem}_{idx}.jpg"
                )
            else:
                # Save to test dir
                cropped_img.save(
                    export_test_dir / f"{int(c)}_c1s1_{label_path.stem}_{idx}.jpg"
                )


800it [00:31, 25.26it/s]


In [7]:
# Zip market data folder and uplaod to gcs

!cd ../data/market1501; zip -r ../market1501.zip *
!gsutil cp ../data/market1501.zip gs://cloud-ai-platform-e8edc327-855c-4911-bb8e-205517f8c899/cv/market1501.zip
!rm ../data/market1501.zip

Copying file://../data/market1501.zip [Content-Type=application/zip]...
| [1 files][ 56.8 MiB/ 56.8 MiB]                                                
Operation completed over 1 objects/56.8 MiB.                                     
